In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold

In [3]:
# preprocess test data
trainset = pd.DataFrame.from_csv('train.csv', index_col=1)
trainset = trainset.fillna('formerlyNA')
trainset[['Upc', 'FinelineNumber']] = trainset[['Upc', 'FinelineNumber']].astype(str)
n = trainset.shape[0]

In [4]:
train_result = trainset['TripType']
train_result = train_result.groupby(by=train_result.index, sort=False).mean()

In [5]:
train_data_not_count = pd.get_dummies(trainset['Weekday'])
train_data_not_count = train_data_not_count.groupby(by=train_data_not_count.index, sort=False).mean()

In [6]:
train_data_count_dep = pd.get_dummies(trainset['DepartmentDescription'])
tmp_index = train_data_count_dep.index
tmp_columns = list(train_data_count_dep.columns.values)
tmp_table = np.array(train_data_count_dep) * np.array(trainset['ScanCount']).reshape((n, 1))
train_data_count_dep = pd.DataFrame(tmp_table)
train_data_count_dep.columns = tmp_columns
train_data_count_dep.index = tmp_index
train_data_count_dep = train_data_count_dep.groupby(by=train_data_count_dep.index, sort=False).sum()

# need to separate between returned and bought goods

In [7]:
train_data_tot_items = trainset['ScanCount'].groupby(by=trainset.index, sort=False).sum()

In [8]:
# fineline_density = trainset['FinelineNumber'].value_counts()
# sparsity = n * 0.003
# print np.sum(fineline_density > sparsity)

In [9]:
train = pd.concat([train_data_not_count, train_data_count_dep, train_data_tot_items], axis=1)

In [10]:
# preprocess test data
testset = pd.DataFrame.from_csv('test.csv', index_col=0)
test_data_not_count = pd.get_dummies(testset['Weekday'])
test_data_not_count = test_data_not_count.groupby(by=test_data_not_count.index, sort=False).mean()
n_test = testset.shape[0]

In [11]:
test_data_count_dep = pd.get_dummies(testset['DepartmentDescription'])
tmp_index = test_data_count_dep.index
tmp_columns = list(test_data_count_dep.columns.values)
tmp_table = np.array(test_data_count_dep) * np.array(testset['ScanCount']).reshape((n_test, 1))
test_data_count_dep = pd.DataFrame(tmp_table)
test_data_count_dep.columns = tmp_columns
test_data_count_dep.index = tmp_index
test_data_count_dep = test_data_count_dep.groupby(by=test_data_count_dep.index, sort=False).sum()

In [12]:
test_data_tot_items = testset['ScanCount'].groupby(by=testset.index, sort=False).sum()

In [13]:
test = pd.concat([test_data_not_count, test_data_count_dep, test_data_tot_items], axis=1)

In [14]:
# Find common coloumns
col_train = list(train.columns.values)
col_test = list(test.columns.values)
col_common = []
# add only common columns for train and test
for col in col_train:
    if col in col_test:
        col_common.append(col)
train = train[col_common]
test = test[col_common]

In [15]:
# Common preprocessing
# Standardizing
stding = StandardScaler()
train = stding.fit_transform(train)
test = stding.transform(test)

In [16]:
classifier = RandomForestClassifier(n_estimators=200)

In [17]:
# # CV n = 4
# cv_n = 4
# kf = KFold(train.shape[0], n_folds=cv_n, shuffle=True)

# metric = []
# for train_index, test_index in kf:
#     X_train, X_test = train[train_index, :], train[test_index, :]
#     y_train, y_test = train_result[train_index].ravel(), train_result[test_index].ravel()

#     # train machine learning
#     classifier.fit(X_train, y_train)

#     # predict
#     class_pred = classifier.predict_proba(X_test)

#     # evaluate
#     metric.append(log_loss(y_test, class_pred))
# print 'The log loss is: ', np.mean(metric)



In [18]:
# predict testset
classifier.fit(train, train_result)
predicted_results = classifier.predict_proba(test)

In [19]:
predicted_results_self = classifier.predict_proba(train)
log_loss(train_result ,predicted_results_self)

0.3211670336447216

In [20]:
# # PCA
# pcaing = PCA(n_components=2)
# train_pca = pcaing.fit_transform(train)
# test_pca = pcaing.transform(test)

In [21]:
submission_file = pd.DataFrame.from_csv("sample_submission.csv")
submission_file[list(submission_file.columns.values)] = predicted_results
submission_file.to_csv("trysub_rf.csv")